In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
import copy
import gzip

custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)
#sns.set(font='Adobe Devanagari')
sns.set_context("paper", font_scale=1, rc={"lines.linewidth": 0.5, "grid.linewidth": 0.3})


matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
# Not italized latex
matplotlib.rcParams['mathtext.default'] = 'regular'
matplotlib.rcParams["font.weight"] = "light"

plt.style.use('themes/rose-pine-moon.mplstyle')


%config InlineBackend.figure_format='svg'

In [2]:
# Load the data
data = pd.read_feather('data/temp/data.feather')

In [3]:
print(len(data))
data.head()


1821316


,date,primaryTime,emulated,name,releaseDate,name_category,name_user,location,platform_name
8,2021-12-22,449.533,False,RE:RUN,2020-08-08,Any%,quebecpower,ca/qc,PC
9,2019-02-09,194.000,False,Time Warpers,2018-11-02,Zones 1-100,_zuR,br,PC
11,2014-11-17,857.000,True,Kirby's Dream Land,1992-04-27,Normal Mode,SapphireYoshi,us,Game Boy
12,2014-11-17,3062.000,False,Kirby: Nightmare in Dream Land,2002-10-25,Any%,SapphireYoshi,us,Game Boy Advance
13,2014-11-17,2454.000,False,Kirby & The Amazing Mirror,2004-04-15,Any%,SapphireYoshi,us,Game Boy Advance


In [4]:
# Find the games ("name") with the most submissions, and keep the top 100, along with their release date and platform
top_games = data[["name", "name_category"]].value_counts()


# Convert the index to columns
top_games = top_games.reset_index()

# For each game, find the first release date and platform
top_games_data = data[data["name"].isin(top_games["name"])].groupby(["name", "name_category"]).agg(
    {"releaseDate": "first", "platform_name": "first"}
)


# Add count of submissions for each game and category in the top games
top_games_data = top_games_data.join(data[["name", "name_category"]].value_counts(), on=["name", "name_category"])


# Add sum of all submissions for each game in the top games
top_games_data["sum_count"] = top_games_data["count"].groupby("name").transform("sum")

# Sort by count of submissions (descending)
top_games_data = top_games_data.sort_values(["sum_count", "count"], ascending=False)

# Remove any rown that have NaN or None
top_games_data = top_games_data.dropna()

# Reset the index
top_games_data = top_games_data.reset_index()

# Keep the 100 unique games (name) with the most submissions, disregarding the category
top_games_data_names = top_games_data.drop_duplicates(subset=["name"]).head(1000)

# Only keep the games that are in the top 100 games with the most submissions
top_games_data = top_games_data[top_games_data["name"].isin(top_games_data_names["name"])]

# Fill NaN values with 0
top_games_data = top_games_data.fillna(0)

top_games_data

,name,name_category,releaseDate,platform_name,count,sum_count
0,Super Mario 64,16 Star,1996-06-23,Nintendo 64,17485,34823
1,Super Mario 64,70 Star,1996-06-23,Nintendo 64,9610,34823
2,Super Mario 64,120 Star,1996-06-23,Nintendo 64,4767,34823
3,Super Mario 64,1 Star,1996-06-23,Nintendo 64,2248,34823
4,Super Mario 64,0 Star,1996-06-23,Nintendo 64,713,34823
...,...,...,...,...,...,...
8041,The Witness Randomizer,Sigma Expert Double,2018-10-27,PC,4,288
8042,Kirby's Return to Dream Land Deluxe,Any% Extra Mode,2023-02-24,Switch,3,288
8043,Kirby's Return to Dream Land Deluxe,Kirby Master,2023-02-24,Switch,3,288
8044,S3AIR - Category Extensions,All Achievements,2019-06-01,PC,3,288


In [5]:
# Only keep the data for the top 100 games
data_top_1000 = data[data["name"].isin(top_games_data["name"])]

# Drop platform_name for this analysis
data_top_1000 = data_top_1000.drop(columns=["platform_name"])

data_top_1000

,date,primaryTime,emulated,name,releaseDate,name_category,name_user,location
8,2021-12-22,449.533,False,RE:RUN,2020-08-08,Any%,quebecpower,ca/qc
11,2014-11-17,857.000,True,Kirby's Dream Land,1992-04-27,Normal Mode,SapphireYoshi,us
12,2014-11-17,3062.000,False,Kirby: Nightmare in Dream Land,2002-10-25,Any%,SapphireYoshi,us
13,2014-11-17,2454.000,False,Kirby & The Amazing Mirror,2004-04-15,Any%,SapphireYoshi,us
14,2015-04-28,2274.000,True,Goof Troop,1993-07-11,Goofy,SapphireYoshi,us
...,...,...,...,...,...,...,...,...
3995704,2022-09-08,1628.340,False,Superliminal,2019-11-12,Glitchless,Valarius,de
3995705,2022-09-17,1537.460,False,Superliminal,2019-11-12,Glitchless,Valarius,de
3995706,2022-10-23,1511.720,False,Superliminal,2019-11-12,Glitchless,Valarius,de
3995722,2020-10-06,306.250,False,"Give Up, Robot",2010-05-06,New Game,TripleHaven,us


In [6]:
# Convert column date to datetime
data_top_1000["date"] = pd.to_datetime(data_top_1000["date"])

# Only keep the data from 2012 onwards
data_top_1000 = data_top_1000[data_top_1000["date"].dt.year >= 2012]

data_top_1000

,date,primaryTime,emulated,name,releaseDate,name_category,name_user,location
8,2021-12-22,449.533,False,RE:RUN,2020-08-08,Any%,quebecpower,ca/qc
11,2014-11-17,857.000,True,Kirby's Dream Land,1992-04-27,Normal Mode,SapphireYoshi,us
12,2014-11-17,3062.000,False,Kirby: Nightmare in Dream Land,2002-10-25,Any%,SapphireYoshi,us
13,2014-11-17,2454.000,False,Kirby & The Amazing Mirror,2004-04-15,Any%,SapphireYoshi,us
14,2015-04-28,2274.000,True,Goof Troop,1993-07-11,Goofy,SapphireYoshi,us
...,...,...,...,...,...,...,...,...
3995704,2022-09-08,1628.340,False,Superliminal,2019-11-12,Glitchless,Valarius,de
3995705,2022-09-17,1537.460,False,Superliminal,2019-11-12,Glitchless,Valarius,de
3995706,2022-10-23,1511.720,False,Superliminal,2019-11-12,Glitchless,Valarius,de
3995722,2020-10-06,306.250,False,"Give Up, Robot",2010-05-06,New Game,TripleHaven,us


In [7]:
dic_score = {1: 10, 2: 5, 3:3, 4:2, 5:1} 

In [8]:
# Assuming data_top_1000 is your DataFrame
# Ensure 'date' is in datetime format and extract the year and month
data_top_1000['date'] = pd.to_datetime(data_top_1000['date'])
data_top_1000['year_month'] = data_top_1000['date'].dt.to_period('M')

# Define the score dictionary
dic_score = {1: 10, 2: 5, 3: 3, 4: 2, 5: 1}

# Sort the dataframe by 'name' (game), 'name_user' (runner), and 'date' (chronologically)
data_top_1000 = data_top_1000.sort_values(by=['name', 'name_category', 'name_user', 'date'])

# Initialize a list to store the results for each month
results = []
l_data_top_100_month = []
# Iterate over each unique month in the dataset
# Sort the months before looping
sorted_months = sorted(data_top_1000['year_month'].unique())

for month in sorted_months:
    # Filter data up to and including the current month
    data_top_1000_month = data_top_1000[data_top_1000['year_month'] <= month].copy()

    # Compute the best time for each runner (lowest primaryTime)
    data_top_1000_month['best_time'] = data_top_1000_month.groupby(['name', 'name_category', 'name_user'])['primaryTime'].transform('min')

    # Remove all the runs that are not the best time for each runner
    data_top_1000_month = data_top_1000_month[data_top_1000_month['primaryTime'] == data_top_1000_month['best_time']]
    
    # Remove any potential duplicates
    data_top_1000_month = data_top_1000_month.drop_duplicates(subset=['name', 'name_category', 'name_user', 'best_time'])

    # Rank runners based on their best time up to this month across all previous submissions
    data_top_1000_month['rank'] = data_top_1000_month.groupby(['name', 'name_category'])['best_time'].rank(method='min', ascending=True)

    # Assign scores based on the rank
    data_top_1000_month['score'] = data_top_1000_month['rank'].map(dic_score).fillna(0)  # fill NaN with 0 if rank > 5

    # Extract country from location
    data_top_1000_month['country'] = data_top_1000_month['location'].str.split('/').str[0]

    l_data_top_100_month.append(data_top_1000_month)

    # Aggregate scores by country for the current month
    country_scores = data_top_1000_month.groupby('country')['score'].sum().reset_index()

    # Add the current month to the result
    country_scores['year_month'] = month

    # Append the result for the current month
    results.append(country_scores)

# Concatenate the results for all months
data_top_1000_final = pd.concat(results)

/var/folders/l8/7h5sfbjn2ng3kg7y8p21r8qr0000gn/T/ipykernel_81046/3626730419.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_top_1000['date'] = pd.to_datetime(data_top_1000['date'])
/var/folders/l8/7h5sfbjn2ng3kg7y8p21r8qr0000gn/T/ipykernel_81046/3626730419.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_top_1000['year_month'] = data_top_1000['date'].dt.to_period('M')


In [9]:
last_month = l_data_top_100_month[-2]
before_last_month = l_data_top_100_month[-18]

In [10]:
data_top_1000_final

,country,score,year_month
0,au,30.0,2012-01
1,ca,15.0,2012-01
2,de,30.0,2012-01
3,fi,10.0,2012-01
4,jp,30.0,2012-01
...,...,...,...
240,ye,33.0,2023-11
241,yt,4.0,2023-11
242,za,86.0,2023-11
243,zm,0.0,2023-11


In [11]:
# Remove the country with score 0
data_top_1000_final = data_top_1000_final[data_top_1000_final["score"] > 0]

data_top_1000_final

,country,score,year_month
0,au,30.0,2012-01
1,ca,15.0,2012-01
2,de,30.0,2012-01
3,fi,10.0,2012-01
4,jp,30.0,2012-01
...,...,...,...
239,xk,20.0,2023-11
240,ye,33.0,2023-11
241,yt,4.0,2023-11
242,za,86.0,2023-11


In [12]:
# Show the top 10 countries with the highest score in the last month
top_countries = data_top_1000_final[data_top_1000_final["year_month"] == sorted_months[-30]].sort_values("score", ascending=False).head(10)
# top_countries

In [13]:
# Convert column year_month to string
data_top_1000_final["year_month"] = data_top_1000_final["year_month"].astype(str)
data = {'header': data_top_1000_final.columns.to_list(), 'data': data_top_1000_final.values.tolist()}

/var/folders/l8/7h5sfbjn2ng3kg7y8p21r8qr0000gn/T/ipykernel_81046/2196133767.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_top_1000_final["year_month"] = data_top_1000_final["year_month"].astype(str)


In [14]:
# Save the graph data as a json file
with open("/Users/cdroin/Library/CloudStorage/GoogleDrive-colasdroin@gmail.com/Mon Drive/Data_viz/speedruns/app/public/data/race_data.json", "w") as f:
    json.dump(data, f)

# Save the graph data as a json file
with gzip.open("/Users/cdroin/Library/CloudStorage/GoogleDrive-colasdroin@gmail.com/Mon Drive/Data_viz/speedruns/app/public/data/race_data.json.gz", "wt") as f:
    json.dump(data, f)